## Assignment Part 1

In [1]:
# install folium
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
# install all libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [253]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [254]:
# initiate Beautiful Soup
soup = BeautifulSoup(data, 'html.parser')

In [255]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [256]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.strip())
        boroughList.append(cells[1].text.strip())
        neighborhoodList.append(cells[2].text.strip()) # avoid new lines in neighborhood cell

In [257]:
# create the dataframe
toronto_df = pd.DataFrame({"Postal Code": postalCodeList, "Borough": boroughList,"Neighborhood": neighborhoodList})
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [258]:
# drop rows having Borough = Not Assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [259]:
# for rows with no neighborhood, put the same name as that of its corresponding borough
for index, row in toronto_df_dropna.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [119]:
# view the dataframe
toronto_df_dropna.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [120]:
# replace / with ,
toronto_df_dropna.replace(r'/',',',regex=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [121]:
# shape of dataframe
toronto_df_dropna.shape

(103, 3)

## Assignment Part 2




In [122]:
# read the Geospatial Data
data2=pd.read_csv("http://cocl.us/Geospatial_data")

In [123]:
data2.head(103)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [124]:
# sort in ascending order by postal code
toronto_df_dropna.sort_values(by=['Postal Code'])

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,Malvern / Rouge
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
18,M1E,Scarborough,Guildwood / Morningside / West Hill
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
44,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
51,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
58,M1N,Scarborough,Birch Cliff / Cliffside West


In [125]:
# re-arrange indexes
toronto_df_dropna.reset_index(drop=True,inplace=True)

In [126]:
toronto_df_dropna.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [260]:
toronto_df_dropna.shape

(103, 3)

In [261]:
# merge the two dataframes
merge = toronto_df_dropna.merge(data2, left_index=True, right_index=True)

In [262]:
merge.head(10)

,Postal Code_x,Borough,Neighborhood,Postal Code_y,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,Regent Park / Harbourfront,M1E,43.763573,-79.188711
3,M6A,North York,Lawrence Manor / Lawrence Heights,M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M1H,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476
6,M1B,Scarborough,Malvern / Rouge,M1K,43.727929,-79.262029
7,M3B,North York,Don Mills,M1L,43.711112,-79.284577
8,M4B,East York,Parkview Hill / Woodbine Gardens,M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848


In [263]:
# drop the column Postal Code_y
merge.drop(['Postal Code_y'], axis=1,inplace=True)

In [264]:
# rename Postal Code_x as Postal Code
merge.rename(columns={"Postal Code_x": "Postal Code"},inplace=True)

In [265]:
merge.shape

(103, 5)

In [266]:
# replace / with , in the final dataset 
merge.replace(r'/',',',regex=True)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1B,Scarborough,"Malvern , Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills,43.711112,-79.284577
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


## Assignment 3




In [267]:
# create a dataframe which contains all the boroughs that have the word 'toronto' in it
toronto=merge[merge.Borough.str.contains('Toronto',case=False)]
toronto.reset_index(drop=True,inplace=True)
toronto.shape

(39, 5)

In [281]:
# view dataframe
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.763573,-79.188711
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975


In [282]:
# drop postal code
toronto.drop(['Postal Code'], axis=1,inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [283]:
# view dataframe
toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Regent Park / Harbourfront,43.763573,-79.188711
1,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.773136,-79.239476
2,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,Downtown Toronto,St. James Town,43.799525,-79.318389
4,East Toronto,The Beaches,43.786947,-79.385975


In [284]:
# view shape
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]))

The dataframe has 4 boroughs and 39 neighborhoods.


In [285]:
# get coordinates of toronto
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [286]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [287]:
# create a dataframe of east toronto
easttoronto_data = toronto[toronto['Borough'] == 'East Toronto'].reset_index(drop=True)
easttoronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.786947,-79.385975
1,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,East Toronto,India Bazaar / The Beaches West,43.704324,-79.388790
3,East Toronto,Studio District,43.657162,-79.378937
4,East Toronto,Business reply mail Processing CentrE,43.688905,-79.554724


In [288]:
# get coordinates of east toronto
address = 'East Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.6247901, -79.3934918.


In [289]:
# create map of East Toronto using latitude and longitude values
map_easttoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(easttoronto_data['Latitude'], easttoronto_data['Longitude'], easttoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_easttoronto)  
    
map_easttoronto

In [290]:
# invoke foursquare

CLIENT_ID = 'MJZRSICI1WUB24HW4G5XJ13OH1HUTKZPHPFMSQTJOGYEFIIK' # your Foursquare ID
CLIENT_SECRET = '2GN1O053KYY4SSEVHWKX2JIBUEOL0R4GD4TQEQMDNNLQNCVR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MJZRSICI1WUB24HW4G5XJ13OH1HUTKZPHPFMSQTJOGYEFIIK
CLIENT_SECRET:2GN1O053KYY4SSEVHWKX2JIBUEOL0R4GD4TQEQMDNNLQNCVR


In [291]:
# get info of first neighborhood in east toronto

easttoronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [292]:
neighborhood_latitude = easttoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = easttoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = easttoronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.7869473, -79.385975.


In [293]:
# get data of venues within a radius of 1000 and limit of 500
LIMIT = 500
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MJZRSICI1WUB24HW4G5XJ13OH1HUTKZPHPFMSQTJOGYEFIIK&client_secret=2GN1O053KYY4SSEVHWKX2JIBUEOL0R4GD4TQEQMDNNLQNCVR&v=20180605&ll=43.7869473,-79.385975&radius=1000&limit=500'

In [294]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92ee8d5fb726001baed9d4'},
 'response': {'headerLocation': 'Bayview Woods - Steeles',
  'headerFullLocation': 'Bayview Woods - Steeles, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 43.79594730900001,
    'lng': -79.37353147269727},
   'sw': {'lat': 43.77794729099999, 'lng': -79.39841852730274}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd396d041b9ef3b799c00e6',
       'name': 'Sun Star Chinese Cuisine 翠景小炒',
       'location': {'address': '636 Finch Avenue East',
        'crossStreet': 'btwn Leslie St and Bayview Av',
        'lat': 43.78791448422642,
        'lng': -79.38123404311649,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7879

In [295]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [296]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sun Star Chinese Cuisine 翠景小炒,Chinese Restaurant,43.787914,-79.381234
1,Kori Sushi,Japanese Restaurant,43.791613,-79.392267
2,TD Canada Trust,Bank,43.788074,-79.380367
3,Harvey's,Restaurant,43.792287,-79.393024
4,TD Canada Trust,Bank,43.791387,-79.392082


In [297]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


In [298]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [299]:
# get neaby venues for east toronto
easttoronto_venues = getNearbyVenues(names=easttoronto_data['Neighborhood'],
                                   latitudes=easttoronto_data['Latitude'],
                                   longitudes=easttoronto_data['Longitude']
                                  )

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Business reply mail Processing CentrE


In [300]:
print(easttoronto_venues.shape)
easttoronto_venues.head()

(182, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
1,The Beaches,43.786947,-79.385975,TD Canada Trust,43.788074,-79.380367,Bank
2,The Beaches,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café
3,The Beaches,43.786947,-79.385975,Kaga Sushi,43.787758,-79.381090,Japanese Restaurant
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [301]:
# group venues by count
easttoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business reply mail Processing CentrE,4,4,4,4,4,4
India Bazaar / The Beaches West,32,32,32,32,32,32
Studio District,100,100,100,100,100,100
The Beaches,4,4,4,4,4,4
The Danforth West / Riverdale,42,42,42,42,42,42


In [302]:
print('There are {} uniques categories.'.format(len(easttoronto_venues['Venue Category'].unique())))

There are 81 uniques categories.


In [303]:
# one hot encoding
easttoronto_onehot = pd.get_dummies(easttoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
easttoronto_onehot['Neighborhood'] = easttoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [easttoronto_onehot.columns[-1]] + list(easttoronto_onehot.columns[:-1])
easttoronto_onehot = easttoronto_onehot[fixed_columns]

easttoronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Beer Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Comic Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Lingerie Store,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Music Venue,Office,Other Great Outdoors,Park,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Sports Bar,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [305]:
easttoronto_onehot.shape

(182, 82)

In [306]:
# group one hot encoded features of east toronto by neighborhood
easttoronto_grouped = easttoronto_onehot.groupby('Neighborhood').mean().reset_index()
easttoronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Beer Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Comic Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Lingerie Store,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Music Venue,Office,Other Great Outdoors,Park,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Sports Bar,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Business reply mail Processing CentrE,0.00,0.00,0.00000,0.00,0.00,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.25,0.00000,0.00000,0.00,0.00,0.000000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.000000,0.00000,0.00,0.00000,0.000000,0.00000,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.00,0.00000,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25000,0.00000,0.25000,0.00,0.00,0.00000,0.00,0.000000,0.25000,0.00000,0.00,0.00,0.00,0.00000,0.00000,0.00,0.0000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.00,0.00,0.00,0.00000
1,India Bazaar / The Beaches West,0.00,0.00,0.00000,0.00,0.00,0.00,0.000000,0.03125,0.00000,0.00,0.00,0.00,0.06250,0.00000,0.00,0.00,0.062500,0.00,0.00,0.00000,0.00,0.09375,0.03125,0.00,0.00,0.03125,0.00,0.00000,0.00000,0.000000,0.03125,0.00,0.03125,0.031250,0.00000,0.0625,0.00,0.00,0.000000,0.03125,0.062500,0.00,0.00000,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03125,0.03125,0.09375,0.00,0.00,0.00000,0.00,0.031250,0.09375,0.03125,0.00,0.00,0.00,0.00000,0.00000,0.00,0.0625,0.00,0.00,0.03125,0.00,0.03125,0.00000,0.00,0.00,0.00,0.00000
2,Studio District,0.01,0.01,0.01000,0.01,0.01,0.01,0.020000,0.00000,0.03000,0.01,0.01,0.00,0.03000,0.00000,0.01,0.09,0.090000,0.01,0.01,0.03000,0.01,0.01000,0.02000,0.02,0.01,0.00000,0.02,0.00000,0.00000,0.010000,0.00000,0.01,0.00000,0.010000,0.00000,0.0100,0.01,0.01,0.000000,0.00000,0.030000,0.03,0.00000,0.01,0.02,0.01000,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01000,0.00000,0.01000,0.01,0.01,0.01000,0.02,0.020000,0.01000,0.01000,0.01,0.01,0.01,0.01000,0.00000,0.01,0.0000,0.01,0.02,0.01000,0.02,0.00000,0.00000,0.01,0.01,0.01,0.00000
3,The Beaches,0.00,0.00,0.00000,0.25,0.00,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.00,0.25000,0.00000,0.25,0.00,0.000000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.000000,0.00000,0.00,0.00000,0.000000,0.00000,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.25,0.00000,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00000,0.00000,0.00000,0.00,0.00,0.00000,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.00,0.00000,0.00000,0.00,0.0000,0.00,0.00,0.00000,0.00,0.00000,0.00000,0.00,0.00,0.00,0.00000
4,The Danforth West / Riverdale,0.00,0.00,0.02381,0.00,0.00,0.00,0.047619,0.02381,0.02381,0.00,0.00,0.00,0.02381,0.02381,0.00,0.00,0.071429,0.00,0.00,0.02381,0.00,0.02381,0.02381,0.00,0.00,0.00000,0.00,0.02381,0.02381,0.047619,0.00000,0.00,0.00000,0.190476,0.02381,0.0000,0.00,0.00,0.047619,0.02381,0.071429,0.00,0.02381,0.00,0.00,0.02381,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00000,0.00000,0.02381,0.00,0.00,0.02381,0.00,0.047619,0.00000,0.00000,0.00,0.00,0.00,0.02381,0.02381,0.00,0.0000,0.00,0.00,0.00000,0.00,0.00000,0.02381,0.00,0.00,0.00,0.02381


In [307]:
easttoronto_grouped.shape

(5, 82)

In [308]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in easttoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = easttoronto_grouped[easttoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business reply mail Processing CentrE----
            venue  freq
0  Sandwich Place  0.25
1        Bus Line  0.25
2     Pizza Place  0.25
3            Park  0.25
4             Pub  0.00


----India Bazaar / The Beaches West----
                venue  freq
0         Pizza Place  0.09
1      Sandwich Place  0.09
2        Dessert Shop  0.09
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Studio District----
                       venue  freq
0             Clothing Store  0.09
1                Coffee Shop  0.09
2         Italian Restaurant  0.03
3        Japanese Restaurant  0.03
4  Middle Eastern Restaurant  0.03


----The Beaches----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                  Pub  0.00


----The Danforth West / Riverdale----
                venue  freq
0    Greek Restaurant  0.19
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3          Resta

In [309]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [310]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = easttoronto_grouped['Neighborhood']

for ind in np.arange(easttoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(easttoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing CentrE,Pizza Place,Sandwich Place,Park,Bus Line,Yoga Studio,Department Store,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market
1,India Bazaar / The Beaches West,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Gym,Seafood Restaurant,Greek Restaurant
2,Studio District,Coffee Shop,Clothing Store,Italian Restaurant,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Fast Food Restaurant,Electronics Store
3,The Beaches,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Furniture / Home Store,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant
4,The Danforth West / Riverdale,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Frozen Yogurt Shop,Juice Bar,Indian Restaurant


In [311]:
# set number of clusters
kclusters = 4

easttoronto_grouped_clustering = easttoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(easttoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 3, 3, 1, 0], dtype=int32)

In [312]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

easttoronto_merged = easttoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
easttoronto_merged = easttoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

easttoronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.786947,-79.385975,1,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Furniture / Home Store,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant
1,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Frozen Yogurt Shop,Juice Bar,Indian Restaurant
2,East Toronto,India Bazaar / The Beaches West,43.704324,-79.388790,3,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Gym,Seafood Restaurant,Greek Restaurant
3,East Toronto,Studio District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Italian Restaurant,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Fast Food Restaurant,Electronics Store
4,East Toronto,Business reply mail Processing CentrE,43.688905,-79.554724,2,Pizza Place,Sandwich Place,Park,Bus Line,Yoga Studio,Department Store,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market


In [313]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(easttoronto_merged['Latitude'], easttoronto_merged['Longitude'], easttoronto_merged['Neighborhood'], easttoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [323]:
# Cluster 0
easttoronto_merged.loc[easttoronto_merged['Cluster Labels'] == 0, easttoronto_merged.columns[[1] + list(range(4, easttoronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,The Danforth West / Riverdale,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Frozen Yogurt Shop,Juice Bar,Indian Restaurant


In [322]:
# Cluster 1
easttoronto_merged.loc[easttoronto_merged['Cluster Labels'] == 1, easttoronto_merged.columns[[1] + list(range(4, easttoronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,1,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Furniture / Home Store,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant


In [321]:
# Cluster 2
easttoronto_merged.loc[easttoronto_merged['Cluster Labels'] == 2, easttoronto_merged.columns[[1] + list(range(4, easttoronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Business reply mail Processing CentrE,2,Pizza Place,Sandwich Place,Park,Bus Line,Yoga Studio,Department Store,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market


In [320]:
# Cluster 3
easttoronto_merged.loc[easttoronto_merged['Cluster Labels'] == 3, easttoronto_merged.columns[[1] + list(range(4, easttoronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,India Bazaar / The Beaches West,3,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Gym,Seafood Restaurant,Greek Restaurant
3,Studio District,3,Coffee Shop,Clothing Store,Italian Restaurant,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Fast Food Restaurant,Electronics Store
